In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from math import *  # 引入 sqrt(), pi, exp 等
import cmath  # 要处理复数情况，用到 cmath.exp()

# Define m1, m2, m3 functions
def m1(kx, ky, t, mu):
    return t * (np.cos(kx) + np.cos(ky)) - mu

def m2(kx, lam):
    return lam * np.sin(kx)

def m3(ky, lam):
    return lam * np.sin(ky)

# Define the Hamiltonian H based on the matrix form
def H(kx, ky, t, mu, lam, c1, c3):
    m1_val = m1(kx, ky, t, mu)
    m2_val = m2(kx, lam)
    m3_val = m3(ky, lam)
    
    return np.array([[c3 * m1_val**2, c3 * m1_val * m2_val, c3 * m1_val * m3_val],
                     [c3 * m1_val * m2_val, c3 * m2_val**2 + c1 * m3_val**2, (c3 - c1) * m2_val * m3_val],
                     [c3 * m1_val * m3_val, (c3 - c1) * m2_val * m3_val, c3 * m3_val**2 + c1 * m2_val**2]])

# Define the commutator [H1, H2]
def commutator(H1, H2):
    return np.dot(H1, H2) - np.dot(H2, H1)

# Define the effective Hamiltonian H_eff using BCH formula (up to the third order)
def H_eff(H1, H2, T1, T2):
    T = T1 + T2
    H1_T1 = T1 * H1
    H2_T2 = T2 * H2
    comm_1_2 = commutator(H1_T1, H2_T2)
    comm_H1_comm_1_2 = commutator(H1_T1, comm_1_2)
    comm_H2_comm_1_2 = commutator(H2_T2, comm_1_2)

    # BCH expansion up to third order
    H_eff_val = (H1_T1 + H2_T2 - (1j / T) * comm_1_2 - (1 / (2 * T**2)) * (comm_H1_comm_1_2 + comm_H2_comm_1_2)) / T
    return H_eff_val

# Define TEI function to construct Hamiltonian H00
def TEI(k, N, c1, c3, t, mu):
    lambda_ = t
    H00 = np.zeros((3 * N, 3 * N), dtype=complex)

    g = t * np.cos(k) - mu
    H0 = [[c3 * (g**2 + t**2 / 2), 0, c3 * g * lambda_ * np.sin(k)],
          [0, c3 * lambda_**2 / 2 + c1 * lambda_**2 * np.sin(k)**2, 0],
          [c3 * g * lambda_ * np.sin(k), 0, c1 * lambda_**2 / 2 + c3 * lambda_**2 * np.sin(k)**2]]

    H1 = [[c3 * (t * g), c3 * g * lambda_ / (2 * 1j), c3 * t * lambda_ * np.sin(k) / 2],
          [c3 * g * lambda_ / (2 * 1j), 0, (c3 - c1) * lambda_**2 * np.sin(k) / (2 * 1j)],
          [c3 * t * lambda_ * np.sin(k) / 2, (c3 - c1) * lambda_**2 * np.sin(k) / (2 * 1j), 0]]

    H2 = [[c3 * t**2 / 4, c3 * t * lambda_ / (4 * 1j), 0],
          [c3 * t * lambda_ / (4 * 1j), c3 * (-lambda_**2) / 4, 0],
          [0, 0, c1 * (-lambda_**2) / 4]]

    for i in range(N):
        H00[i * 3 + 0:i * 3 + 3, i * 3 + 0:i * 3 + 3] = H0

    for i in range(N - 1):
        H00[i * 3 + 0:i * 3 + 3, (i + 1) * 3 + 0:(i + 1) * 3 + 3] = H1
        H00[(i + 1) * 3 + 0:(i + 1) * 3 + 3, i * 3 + 0:i * 3 + 3] = np.conj(np.transpose(H1))

    for i in range(N - 2):
        H00[i * 3 + 0:i * 3 + 3, (i + 2) * 3 + 0:(i + 2) * 3 + 3] = H2
        H00[(i + 2) * 3 + 0:(i + 2) * 3 + 3, i * 3 + 0:i * 3 + 3] = np.conj(np.transpose(H2))

    return H00

# Plot the 3D band structure for the effective Hamiltonian
def plot_3d_band_structure(kx_vals, ky_vals, t1, t2, mu1, mu2, lam1, lam2, c1, c3, c33, T1, T2, ax):
    energies_1 = np.zeros((len(kx_vals), len(ky_vals)))
    energies_2 = np.zeros((len(kx_vals), len(ky_vals)))
    energies_3 = np.zeros((len(kx_vals), len(ky_vals)))

    for i, kx in enumerate(kx_vals):
        for j, ky in enumerate(ky_vals):
            H1 = H(kx, ky, t1, mu1, lam1, c1, c3)
            H2 = H(kx, ky, t2, mu2, lam2, c1, c33)
            H_eff_k = H_eff(H1, H2, T1, T2)
            eigvals = np.linalg.eigvalsh(H_eff_k)  # Get eigenvalues of H_eff
            energies_1[i, j], energies_2[i, j], energies_3[i, j] = np.sort(eigvals)

    KX, KY = np.meshgrid(kx_vals, ky_vals)
    
    # Plot each energy band on the given ax
    ax.plot_surface(KX, KY, energies_1, cmap='viridis', edgecolor='none', alpha=0.7)
    ax.plot_surface(KX, KY, energies_2, cmap='plasma', edgecolor='none', alpha=0.7)
    ax.plot_surface(KX, KY, energies_3, cmap='cool', edgecolor='none', alpha=0.7)
    
    ax.set_xlabel('$k_x$')
    ax.set_ylabel('$k_y$')
    ax.set_zlabel('Energy')

# Plot the 2D band structure for the effective Hamiltonian
def plot_band_structure(k_vals, N, c1, c3, c33, mu1, mu2, T1, T2, ax):
    energies = np.zeros((len(k_vals), 3 * N))

    for i, k in enumerate(k_vals):
        H1 = TEI(k, N, c1, c3, t1, mu1)
        H2 = TEI(k, N, c1, c33, t2, mu2)
        H_eff_k = H_eff(H1, H2, T1, T2)
        eigvals = np.linalg.eigvalsh(H_eff_k)  # Get all eigenvalues
        energies[i, :] = np.sort(np.real(eigvals))

    for band in range(3 * N):
        ax.plot(k_vals, energies[:, band], color='b')

    ax.set_xlabel('$k$')
    ax.set_ylabel('Energy')
    #ax.set_title('Band Structure of Effective Hamiltonian $H_{eff}$')

# Define parameters
kx_vals = np.linspace(-np.pi, np.pi, 50)
ky_vals = np.linspace(-np.pi, np.pi, 50)
k_vals = np.linspace(-pi, pi, 300)
N = 40

# System parameters
t1 = t2 = 1.0
mu1 = 1
mu2 = -1
lam1 = lam2 = 1.0
c1 = 0.25
c3 = c33 = 1
T1 = T2 = 1

# Create interactive widgets for mu1, mu2, T1, and T2
@interact(mu1=(-4, 4, 1), mu2=(-4, 4, 1), T1=(1, 100, 10), T2=(0, 100, 10))
def interactive_plot(mu1=mu1, mu2=mu2, T1=T1, T2=T2):
    fig = plt.figure(figsize=(14, 7))

    # Plot 3D band structure
    ax1 = fig.add_subplot(1, 2, 1, projection='3d', azim=0, elev=0)
    plot_3d_band_structure(kx_vals, ky_vals, t1, t2, mu1, mu2, lam1, lam
                           2, c1, c3, c33, T1, T2, ax1)

    # Plot 2D band structure
    ax2 = fig.add_subplot(1, 2, 2)
    plot_band_structure(k_vals, N, c1, c3, c33, mu1, mu2, T1, T2, ax2)

    # Adjust layout to avoid overlap
    plt.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=1, description='mu1', max=4, min=-4), IntSlider(value=-1, description='m…